# Karpathy's nano GPT implementation sandox notebook

In [13]:
import torch
from torch.nn import functional as F
import torch.nn as nn

In [ ]:
# Init
torch.manual_seed(4242)
B,T,C = 3,9,32 # batch, time, channels
x = torch.randn(B,T,C)

## Naive implementation of matrix multiplication

In [ ]:
xbow = torch.zeros((B,T,C))
for b in range(B):
    for t in range(T):
        xprev = x[b, :t+1] # (t,C)
        xbow[b,t] = torch.mean(xprev, 0) # (C)

## Faster implementation of matrix multiplication using tril

In [ ]:
#
wei = torch.tril(torch.ones((T,T)))
wei = wei / wei.sum(1, keepdim=True)
xbow2 = wei @ x # (B,T,T) @ (B,T,C) --> (B,T,C)
#
torch.allclose(xbow, xbow2)

## Implementing self-attention

In [17]:
head_size = 16
key = nn.Linear(in_features=C, out_features=head_size, bias=False)
query = nn.Linear(in_features=C, out_features=head_size, bias=False)
value = nn.Linear(in_features=C, out_features=head_size, bias=False)
k = key(x) # (B,T,C) -> (B,T,16)
q = query(x) # (B,T,C) -> (B,T,16)
wei = q @ k.transpose(-2,-1) # (B, T, 16) @ (B, 16, T) -> (B, T, T)

tril = torch.tril(torch.ones((T,T)))
#wei = torch.zeros((T,T))
wei = wei.masked_fill(tril == 0, float('-inf'))
wei = F.softmax(wei, dim=-1)
v = value(x) # (B,T,C) -> (B,T,16)
out = wei @ v

out.shape

torch.Size([3, 9, 16])